In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
import random
from statsmodels.stats.power import tt_ind_solve_power

def date_converter(x):
    return datetime.datetime.strptime(x,'%Y-%m-%d')

### Задание 3
**Срок сдачи: 07 апреля 18:00**

В этом задание вы будете работать с выручкой - метрикой с достаточно высокой дисперсией и соответственно низкой чувствительностью к изменениям.

Вам предстоит разработать подход по повышению чувствительности этой метрики.
От успеха данной инициативы зависит то, как быстро мы сможем проверять продуктовые гипотезы.

#### Для планирования эксперимента изучите данные и метрики 2 балла:

- Загрузите файл и изучите данные;
- Изучите основные статистики метрики: среднее, стандартное отклонение;
- Изучите то, как эти статистики изменяются внутри срезов: user_segment, region, category

#### Предскажите минимально детектируемый эффект (MDE) 2 балла:

- Для размеров тестовых групп 10,25,50% и длительностей 30, 60, 90 дней предскажите минимально детектируемый эффект;


#### CUPED 6 баллов
Разработайте подход по снижению дисперсии на основе методики CUPED:

- Рассчитайте CUPED метрику на основе данных за период в 30 дней перед предполагаемым периодом сбора основной выборки и опишите как изменился MDE для выборок 50/50%, собранных за 30 дней (2 балла). 
- Проверьте насколько данный подход подвержен сезональности. Как меняется MDE, если применять данный подход в разные периоды (2 балла). 
- Оптимизируйте подход варьируя количество дней до теста, которые вы будете для расчёта CUPED метрики. Какое максимальное снижение MDE вы смогли достичь (2 балла).



#### * Стратификация 4 балла (задание для получения дополнительных баллов)
Разработайте подход по снижению дисперсии на основе методики стратификации выборок:

- Стратифицируйте выборки внутри срезов user_segment, region, category
- Рассчитайте стратифицированные среднее,стандартное отклонение для этих выборок и MDE для теста 50/50% длительностью 30 дней.
- В каких срезах достигается наибольшее снижение MDE?



#### Описание данныx

event_date - дата 

user_id - идентификатор пользователя

user_segment - сегмент пользователей

region - регион

category - категория авито

revenue_amount - сумма покупок


# Задание 1

In [3]:
df = pd.read_csv('user_amount_var_reduction.csv')
df['event_date']=df.event_date.apply(date_converter)

In [4]:
df

,event_date,user_id,user_segment,category,region,revenue_amount
0,2019-06-18,4129257975403696,4626944681007198896,7204754148424990691,2201388710798473628,139
1,2019-06-18,7225254357470025,4626944681007198896,57514010642945023,3048517866726460689,193
2,2019-06-18,8912846500600569,4626944681007198896,5983323600580891431,3048517866726460689,160
3,2019-06-18,20910801279823046,4626944681007198896,6847326422676498281,7762196236856042106,209
4,2019-06-18,34873300821657220,2376074197230840906,3187769798308634693,3855810523611026650,160
...,...,...,...,...,...,...
8625837,2019-12-08,9170823997728769058,2376074197230840906,4658595883120885993,3855810523611026650,133
8625838,2019-12-08,9178869387857307556,4626944681007198896,1472231361867825698,3048517866726460689,63
8625839,2019-12-08,9186690404771258365,2376074197230840906,4546178272531746227,8867014108346120905,168
8625840,2019-12-08,9195678548159495844,2376074197230840906,5983323600580891431,1187552825302355021,118


In [13]:
print(f'Mean of the revenue is {df.revenue_amount.mean():.2f}')
print(f'Std is {df.revenue_amount.std():.2f}')

Mean of the revenue is 400.90
Std is 960.22


In [38]:
df_segments_mean = df.groupby(['user_segment'])['revenue_amount'].mean()
df_segments_mean.reset_index()
df_segments_std = df.groupby(['user_segment'])['revenue_amount'].std()
df_segments_std.reset_index()
df_segments = pd.merge(df_segments_mean, df_segments_std, how="right", on=["user_segment"])
df_segments.rename(columns = {'revenue_amount_x': 'mean', 'revenue_amount_y': 'std'}, inplace = True)
df_segments

,mean,std
user_segment,,
853431062533847667,1094.330305,2392.330098
2376074197230840906,401.615956,571.446013
4626944681007198896,207.760172,285.545686


In [37]:
df_category_mean = df.groupby(['category'])['revenue_amount'].mean()
df_category_mean.reset_index()
df_category_std = df.groupby(['category'])['revenue_amount'].std()
df_category_std.reset_index()
df_category = pd.merge(df_category_mean, df_category_std, how="right", on=["category"])
df_category.rename(columns = {'revenue_amount_x':'mean', 'revenue_amount_y':'std'}, inplace = True)
df_category

,mean,std
category,,
57514010642945023,347.826167,605.864274
91460430793700892,441.861699,651.938097
174490086998335078,325.149759,764.813512
264558924403680107,509.373364,747.222091
282115902545472456,260.778575,354.734991
320230973495659885,286.013254,414.997017
1472231361867825698,258.353200,594.984151
1560592244484430230,313.285665,464.354426
1793067034829450750,497.431001,1073.795206


In [39]:
df_region_mean = df.groupby(['region'])['revenue_amount'].mean()
df_region_mean.reset_index()
df_region_std = df.groupby(['region'])['revenue_amount'].std()
df_region_std.reset_index()
df_region = pd.merge(df_region_mean, df_region_std, how="right", on=["region"])
df_region.rename(columns = {'revenue_amount_x':'mean', 'revenue_amount_y':'std'}, inplace = True)
df_region

,mean,std
region,,
68475388295793322,289.345317,504.681673
181387893390186210,299.629748,461.930430
308194675497345788,252.626919,544.986118
385667469633138042,272.589534,507.917500
405364780634206711,205.209615,295.197711
...,...,...
9030439280442565223,241.213507,374.722208
9032475813655386611,341.125571,583.562907
9159753098538066161,324.889870,721.907652


# Задание 2

Для размеров тестовых групп 10, 25, 50% и длительностей 30, 60, 90 дней предскажите минимально детектируемый эффект;

In [62]:
def choose_date(data, start_date, delta, share): 
    tmp = data[(data.event_date >= start_date) & (data.event_date <= start_date + datetime.timedelta(days=delta))]
    tmp = tmp.groupby(['user_id']).sum().reset_index()
    x = tmp[(tmp.user_id) % 100 <= int(100 * share)].reset_index()
    return x

In [63]:
from statsmodels.stats.power import tt_ind_solve_power

def mde(size: int):
    for duration in range(30, 91, 30):
        
        x1 = choose_date(df, min(df.event_date), duration, size/100) #получаем необходимую подвыборку
 
        mean = x1.revenue_amount.mean()
        std = x1.revenue_amount.std()
 
        alpha = 0.01
        power = 0.8
        result_effect = 0.04 # целевой размер относительного отклонения исходной метрики
        effect_size = (mean /std) * result_effect
 
    
        sample_vol = tt_ind_solve_power(effect_size=effect_size, 
                                        alpha=alpha,
                                        power=power, 
                                        nobs1=None, 
                                        ratio=1, 
                                        alternative = 'two-sided')    
      
        mde = tt_ind_solve_power(effect_size = None, 
                                alpha = alpha,
                                power = power, 
                                nobs1 = sample_vol, 
                                ratio = 1,  
                                alternative = 'two-sided')

        print(f'Duration={duration}, size={size/100}, mde={round(mde, 3)}')

In [64]:
print('Minimum detectable effect size')
mde(10)
mde(25)
mde(50)

Minimum detectable effect size
Duration=30, size=0.1, mde=0.006
Duration=60, size=0.1, mde=0.005
Duration=90, size=0.1, mde=0.004
Duration=30, size=0.25, mde=0.005
Duration=60, size=0.25, mde=0.004
Duration=90, size=0.25, mde=0.004
Duration=30, size=0.5, mde=0.006
Duration=60, size=0.5, mde=0.005
Duration=90, size=0.5, mde=0.004


# Задание 3

Рассчитайте CUPED метрику на основе данных за период в 30 дней перед предполагаемым периодом сбора основной выборки и опишите как изменился MDE для выборок 50/50%, собранных за 30 дней (2 балла).

In [66]:
df.event_date.min()

Timestamp('2019-06-18 00:00:00')

In [86]:
pre_month = df[df.event_date < '2019-07-18']
pre_month = pre_month.groupby(['user_id'])['revenue_amount'].mean().reset_index()
curr_month = df[(df.event_date >= '2019-07-18') & (df.event_date < '2019-08-18')]
curr_month = curr_month.groupby(['user_id'])['revenue_amount'].mean().reset_index()
print(pre_month.shape)
print(curr_month.shape)

(606264, 2)
(599033, 2)


In [90]:
df_new = pd.merge(pre_month, curr_month, on='user_id', how='left')[['user_id', 'revenue_amount_x', 'revenue_amount_y']]
df_new['revenue_amount_y'] = df_new['revenue_amount_y'].fillna(0)
df_new.rename(columns = {'revenue_amount_x':'prev_revenue', 'revenue_amount_y':'curr_revenue'}, inplace = True)
df_new

,user_id,prev_revenue,curr_revenue
0,1336907904656,660.5,0.000000
1,1912219714366,70.0,100.333333
2,7562887359106,52.5,0.000000
3,23708868092799,55.0,48.000000
4,35353400632087,209.0,0.000000
...,...,...,...
606259,9223325950201039894,453.0,0.000000
606260,9223348554567144516,27.5,49.000000
606261,9223359827025968255,48.0,0.000000
606262,9223361693694781577,195.0,0.000000


In [92]:
teta = np.cov(df_new.curr_revenue, df_new.prev_revenue, ddof=1)[1,0]/np.var(df_new.prev_revenue, ddof=1)

In [93]:
corr = np.corrcoef(df_new.curr_revenue, df_new.prev_revenue)[1,0] 

Считаем CUPED метрику

In [94]:
df_new['cuped_revenue'] = df_new.curr_revenue - teta * (df_new.prev_revenue - df_new.prev_revenue)